In [14]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [15]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os.path as op
from pprint import pprint as pp
import scipy.stats
import seaborn as sns
import sys
pth = op.abspath("../")
print pth
sys.path.insert(0, pth)
import teigen
reload(teigen)
print teigen.__path__

/Users/mjirik/projects/teigen
['/Users/mjirik/projects/teigen/teigen']


In [16]:
tg = teigen.tgmain.Teigen()
config = tg.get_default_config()

In [17]:
# pp(config)
config

{'appearance': {'noise_preview': False,
  'show_aposteriori_surface': True,
  'skip_volume_generation': False},
 'areasampling': {'areasize_mm': [110.0, 100.0, 100.0],
  'areasize_px': [110, 100, 100],
  'voxelsize_mm': [1.0, 1.0, 1.0]},
 'filepattern': '~/teigen_data/{seriesn:03d}/data{:06d}.jpg',
 'generator_id': 0,
 'generators': OrderedDict([('Voronoi tubes',
               OrderedDict([('element_number', 30),
                            ('uniform_radius_distribution', True),
                            ('normal_radius_distribution', False),
                            ('fixed_radius_distribution', False),
                            ('radius_distribution_minimum', 2.0),
                            ('radius_distribution_maximum', 10.0),
                            ('radius_distribution_mean', 5.0),
                            ('radius_distribution_standard_deviation', 5.0),
                            ('random_generator_seed', 0)])),
              ('Gensei', OrderedDict([('n_object

In [18]:
np.arange(0, 10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

# Prepare multiple configs


In [19]:
configs = []
tg = teigen.tgmain.Teigen()

In [20]:
# for resolution in [5, 6, 7, 8, 10, 12, 14, 16, 20, 24, 28, 32, 40, 48]:
#     for rnd in [101, 102, 103, 104, 105]:
#         config = tg.get_default_config()
#         config["generator_id"] = 3
#         config["output"]["note"] = "resolution sphere"
#         gc = config["generators"]["Unconnected tubes"]
#         gc["element_number"] = 10
#         gc["orientation_anisotropic"] = False
#         gc["length_distribution_mean"] = 0.0
#         gc["length_distribution_standard_deviation"] = 0.0
#         gc["random_generator_seed"] = rnd
#         config["postprocessing"]["measurement_resolution"] = resolution
#         config["appearance"]["skip_volume_generation"] = True
#         configs.append(config)

for resolution in [5, 6, 7, 8, 10, 12, 14, 16, 20, 24, 28, 32, 40, 48]:
    for rnd in [101, 102, 103, 104, 105]:
        config = tg.get_default_config()
        config["generator_id"] = 3
        config["output"]["note"] = "resolution tube"
        gc = config["generators"]["Unconnected tubes"]
        gc["element_number"] = 10
        gc["orientation_anisotropic"] = False
        gc["length_distribution_mean"] = 40.0
        gc["length_distribution_standard_deviation"] = 2.0
        gc["random_generator_seed"] = rnd
        config["postprocessing"]["measurement_resolution"] = resolution
        config["appearance"]["skip_volume_generation"] = True
        configs.append(config)


In [21]:

for length in [5, 6, 7, 8, 10, 12, 14, 16, 20, 24, 28, 32, 40, 48]:
    for rnd in [101, 102, 103]:
        config = tg.get_default_config()
        config["generator_id"] = 3
        config["output"]["note"] = "tube length"
        gc = config["generators"]["Unconnected tubes"]
        gc["element_number"] = 10
        gc["orientation_anisotropic"] = False
        gc["length_distribution_mean"] = length
        gc["length_distribution_standard_deviation"] = 0.0
        gc["random_generator_seed"] = rnd
        config["postprocessing"]["measurement_resolution"] = 20
        config["appearance"]["skip_volume_generation"] = True
        configs.append(config)

# Run multiple configs

In [ ]:
for i, cfg in enumerate(configs):
    
    tg = teigen.tgmain.Teigen()
    # config = tg.get_default_config()
    tg.update_config(**cfg)
    print (str(i) + " / " + str(len(configs)))  
    print(gc["random_generator_seed"])
    print(config["postprocessing"]["measurement_resolution"])
    
    tg.run()
    print("run finished")
    tg.step2()
    print("step2 finished")
    del(tg)
    

0 / 112
103
20
41.3022358959
39.3613639108
40.3815886447
43.9575146482
39.7303185588
40.7810556855
39.7664533671
43.8035095905
39.9368417122
41.299651667
40.7720606242
44.1680370607
40.3742490435
38.5343097049
39.6749305305
40.9716174749
39.2176867456
42.0565863216
38.4573427905
40.2209548223
41.5666321016
38.5820925246
38.935058949
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
run finished
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
step2 finished
1 / 112
103
20
38.9803100642
38.1938021629
39.0711120219
40.1019596787
41.9943793653
46.8571258677
41.6039846184
38.856054276
39.9031925577
40.4687286161
39.0429195385
39.2812753321
39.1411939249
39.5536608285
38.5916134818
39.5395220779
39.6271767289
40.1175458548
36.4420280927
41.0587744769
41.0660338588
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
run finished
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.1
step2 finished
2 / 112
103
20
40.8615859064
40.8646315445
41.3605713418
42.6583498005
38.104471664
36.3132361247
40.4974513447
43.4306243055
41.1873767904
3